# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.
%load_ext dotenv
%dotenv 

In [2]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [3]:
import os
from glob import glob

# Write your code below.
PRICE_DATA = os.getenv("PRICE_DATA")
parquet_files = glob(os.path.join(PRICE_DATA, "**/*.parquet"), recursive = True)
parquet_files

['../../05_src/data/prices/CTAS/CTAS_2002/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2005/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2004/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2003/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2010/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2017/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2021/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2019/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2018/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2020/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2016/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2011/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2008/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2006/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2001/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2000/part.0.parquet',
 '../../05_src/data/prices/CTAS/CTAS_2007/part.0.parquet

In [4]:
len(parquet_files)

13078

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [5]:
dd_px = dd.read_parquet(parquet_files)
dd_px


,Date,Adj Close,Close,High,Low,Open,Volume,Year
npartitions=13078,,,,,,,,
,datetime64[ns],float64,float64,float64,float64,float64,float64,int32
,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...


In [6]:
dd_shift = dd_px.groupby('Ticker', group_keys=False).apply(
    lambda x: x.assign(
        Close_lag_1=x['Close'].shift(1),
        Adj_Close_lag_1=x['Adj Close'].shift(1),
    )
)

dd_feat= dd_shift.assign(
    Returns = lambda x: x['Close']/x['Close_lag_1'] - 1,
    Hi_Lo_Range = lambda x: x['High'] - x['Low'] 
)


/var/folders/fz/zg9vlgxj4mnb4rs9hqyrr2180000gn/T/ipykernel_5127/1190984444.py:1: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_shift = dd_px.groupby('Ticker', group_keys=False).apply(


In [7]:
dd_feat

,Date,Adj Close,Close,High,Low,Open,Volume,Year,Close_lag_1,Adj_Close_lag_1,Returns,Hi_Lo_Range
npartitions=13078,,,,,,,,,,,,
,datetime64[ns],float64,float64,float64,float64,float64,float64,int32,float64,float64,float64,float64
,...,...,...,...,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...
,...,...,...,...,...,...,...,...,...,...,...,...


+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [14]:
df = dd_feat.compute()

In [17]:
df['Rolling_10_Mean'] = df['Returns'].rolling(10).mean()

In [18]:
df.head(20)

Price,Date,Adj Close,Close,High,Low,Open,Volume,Year,Close_lag_1,Adj_Close_lag_1,Returns,Hi_Lo_Range,Rolling_10_Mean
Ticker,,,,,,,,,,,,,
DOV,2002-01-02,16.045650,24.842640,24.963299,24.420326,24.869452,1022622.0,2002,NaN,NaN,NaN,0.542973,NaN
DOV,2002-01-03,16.340069,25.298468,25.405722,24.835936,24.876156,989355.0,2002,24.842640,16.045650,0.018349,0.569786,NaN
DOV,2002-01-04,16.721077,25.888363,25.955397,25.405722,25.405722,1319488.0,2002,25.298468,16.340069,0.023317,0.549675,NaN
DOV,2002-01-07,16.608494,25.714075,26.136387,25.687262,25.975506,1060812.0,2002,25.888363,16.721077,-0.006732,0.449125,NaN
DOV,2002-01-08,16.236153,25.137587,25.807922,24.869452,25.673855,976377.0,2002,25.714075,16.608494,-0.022419,0.938471,NaN
DOV,2002-01-09,16.279448,25.204620,25.673855,25.144291,25.204620,745746.0,2002,25.137587,16.236153,0.002667,0.529564,NaN
DOV,2002-01-10,15.963390,24.715275,25.204620,24.400217,25.204620,678317.0,2002,25.204620,16.279448,-0.019415,0.804403,NaN
DOV,2002-01-11,15.859478,24.554394,24.956596,24.534285,24.768902,544056.0,2002,24.715275,15.963390,-0.006509,0.422312,NaN
DOV,2002-01-14,15.595367,24.145491,24.675055,24.138786,24.601318,493335.0,2002,24.554394,15.859478,-0.016653,0.536268,NaN


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

- not necessary, both dask and pandas can calculate the moving average return.
- pandas is great for smaller datasets due to its ease of use and rich functionality. dask is ideal for large datasets as it leverages parallel and distributed computing, allowing for efficient scaling across multiple cores or machines with minimal coordination required.

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.